In [4]:
%cd ~/REVIVAL2
%load_ext autoreload
%autoreload 2
# %load_ext blackcellmagic

/disk2/fli/REVIVAL2
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import sys
import os
import numpy
import openmm
print(f"Python: {sys.version}")
print(f"NumPy: {numpy.__version__}")
print(f"OpenMM: {openmm.version.version}")
print(f"Environment: {os.environ}")

Python: 3.9.7 | packaged by conda-forge | (default, Sep 29 2021, 19:23:11) 
[GCC 9.4.0]
NumPy: 2.0.2
OpenMM: 7.7.0.dev-130124a
Environment: environ({'USER': 'fli', 'SSH_CLIENT': '131.215.248.58 49974 22', 'XDG_SESSION_TYPE': 'tty', 'SHLVL': '2', 'MOTD_SHOWN': 'pam', 'HOME': '/disk2/fli', 'SSL_CERT_FILE': '/usr/lib/ssl/certs/ca-certificates.crt', 'DBUS_SESSION_BUS_ADDRESS': 'unix:path=/run/user/1002/bus', 'LOGNAME': 'fli', '_': '/disk2/fli/miniconda3/envs/vina/bin/python', 'XDG_SESSION_CLASS': 'user', 'XDG_SESSION_ID': '8448', 'VSCODE_CLI_REQUIRE_TOKEN': 'e2fff225-9583-46c0-8e69-bfcf24b4c518', 'PATH': '/disk2/fli/miniconda3/envs/vina/bin:/disk2/fli/.vscode-server/cli/servers/Stable-fabdb6a30b49f79a7aba0f2ad9df9b399473380f/server/bin/remote-cli:/disk2/fli/.local/bin:/disk2/fli/orca_6_0_1:/disk2/fli/miniconda3/envs/vina/bin:/disk2/fli/miniconda3/condabin:/disk2/fli/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin', 'VSCODE_AGENT

In [ ]:
"""
Test the vina docking and extracting score.
"""

import sys
import os


from datetime import datetime

from REVIVAL.zs.vina import dock_lib
from REVIVAL.util import checkNgen_folder

if __name__ == "__main__":

    os.environ["CUDA_VISIBLE_DEVICES"] = "1"

    # log outputs
    f = open(
        os.path.join(
            checkNgen_folder("logs/zs/vina/dock"),
            f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}.out",
        ),
        "w",
    )
    sys.stdout = f

    for chai_dir in [ 
        "zs/chai/mut_structure/PfTrpB-5chloro-plp", 
        "zs/chai/mut_structure/PfTrpB-4cyano-plp",
        "zs/chai/mut_structure/PfTrpB-5chloro-plp",
        "zs/chai/mut_structure/PfTrpB-6chloro-plp",
        "zs/chai/mut_structure/PfTrpB-7bromo-plp",
        "zs/chai/mut_structure/PfTrpB-7indo-plp",
        "zs/chai/mut_structure/PfTrpB-7methyl-plp",
        "zs/chai/mut_structure/PfTrpB-56chloro-plp",
        "zs/chai/mut_structure/PfTrpB-5cyano-plp",
    ]:
        dock_lib(chai_dir, "inactivated-cofactor")


    # def dock_lib(
    #     chai_dir: str,
    #     cofactor_type: str,
    #     vina_dir: str = "vina",
    #     pH: float = 7.4,
    #     method='vina',
    #     size_x=15.0,
    #         size_y=15.0, 
    #         size_z=15.0,
    #         num_modes=9, # Dunno check vina docks using the defaut
    #         exhaustiveness=32 
    # ):

    f.close()

In [5]:

def extract_lowest_energy(file_path: str):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    for i, line in enumerate(lines):
        if "mode |   affinity | dist from best mode" in line:
            table_start = i + 3  # Skip to the actual table
            break
    
    energies = []
    for line in lines[table_start:]:
        if line.strip() == "":  # End of the table
            break
        parts = line.split()
        try:
            energies.append(float(parts[1]))  # Affinity is in the second column
        except ValueError:
            pass  # Skip invalid lines
    
    return min(energies)


In [6]:
extract_lowest_energy(file_path="/disk2/fli/REVIVAL2/zs/vina/PfTrpB-4cyano_inactivated-cofactor/I165A:I183A:Y301V_0/I165A:I183A:Y301V_0-4cyano_log.txt")

-11.81

In [22]:
def extract_ions(input_file_path, output_file_path, ion):
    """
    Extract lines starting with ATOM containing the specified ion from a PDB or CIF file.

    Args:
        input_file_path (str): Path to the input PDB or CIF file.
        output_file_path (str): Path to save the extracted ion lines.
        ion (str): Name of the ion to extract (e.g., "Na", "Cl").
    """
    is_cif = input_file_path.endswith(".cif")
    ion_found = False

    with open(input_file_path, 'r') as infile, open(output_file_path, 'w') as outfile:
        for line in infile:
            if line.startswith("ATOM") or line.startswith("HETATM"):
                fields = line.split()
                if len(fields) >= 4:
                    atom_name = fields[2].strip()  # Atom name
                    residue_name = fields[3].strip()  # Residue name
                    if atom_name.upper() == ion.upper() or residue_name.upper() == ion.upper():
                        outfile.write(line)
                        ion_found = True

    if not ion_found:
        print(f"No ions matching '{ion}' were found in the file.")
    else:
        print(f"Ions matching '{ion}' were successfully extracted to {output_file_path}.")


In [23]:
extract_ions(
    input_file_path="/disk2/fli/REVIVAL2/sandbox/files/WT_0/WT_0.cif",
    output_file_path="/disk2/fli/REVIVAL2/sandbox/files/WT_0/Na.pdb",
    ion="Na"
)

Ions matching 'Na' were successfully extracted to /disk2/fli/REVIVAL2/sandbox/files/WT_0/Na.pdb.
